# Noisy 1B5 (Finetune Training)
We will be finetuning a 1B5 model, and intentionally overtrain it.
This is to measure how the introduction of a noisy state will impact train.

The goal of this experiment, is to measure two seperate objective
- test for overfitting conditions across multiple epoch
- test if overfitting is mitigated with the introduction of a noisy state

> This project assumes you have the rwkv-infctx conda env setup, and you are executing in that environment - see the main README.md for the conda env setup steps

In [ ]:
# First lets get the RWKV base model which we will be finetuning
!mkdir -p ../../../model/
!mkdir -p ../../../datapath/
!mkdir -p ../../../checkpoint/
!cd ../../../model/ && wget https://huggingface.co/picocreator/memory-size-experiment-for-rwkv/resolve/main/Echo-A-1B5-Init.pth

In [ ]:
DEEPSPEED_STRAT="deepspeed_stage_1"
GPU_DEVICES="auto"
ENABLE_WANDB=True
WANDB_PREFIX="MultiEpoch-1B5"

print("DEEPSPEED_STRAT:", DEEPSPEED_STRAT)
print("ENABLE_WANDB:", ENABLE_WANDB)
print("GPU_DEVICES:", GPU_DEVICES)

if ENABLE_WANDB:
    WANDB_MODE="online"
else:
    WANDB_MODE="disabled"

# Computing the notebook, and various paths
import os
NOTEBOOK_DIR=os.path.dirname(os.path.abspath("__file__"))
PROJECT_DIR=os.path.abspath(os.path.join(NOTEBOOK_DIR, "../../../"))
TRAINER_DIR=os.path.abspath(os.path.join(PROJECT_DIR, "./RWKV-v4neo/"))

print("NOTEBOOK_DIR:", NOTEBOOK_DIR)
print("TRAINER_DIR:", TRAINER_DIR)
print("PROJECT_DIR:", PROJECT_DIR)

In [ ]:
# Lets preload the requried dataset
!cd "{TRAINER_DIR}" && \
    python3 preload_dataset.py "{NOTEBOOK_DIR}/MultiEpoch-1B5-shakespeare-baseline.yaml"

## Stage 1 : Baseline model training

In [ ]:
# Start the foundation model training
!cd "{TRAINER_DIR}" && \
    export WANDB_MODE="{WANDB_MODE}" && \
    python new_train.py fit \
        -c "{NOTEBOOK_DIR}/MultiEpoch-1B5-shakespeare-baseline.yaml" \
        --trainer.logger.init_args.name="{WANDB_PREFIX} - Shakespeare Baseline (ctx=2048, {DEEPSPEED_STRAT})" \
        --trainer.strategy="{DEEPSPEED_STRAT}" \
        --trainer.devices="{GPU_DEVICES}" 

In [ ]:
# Lets export the model from the checkpoint
!cd "{TRAINER_DIR}" && \
    python export_checkpoint.py "../checkpoint/MultiEpoch-1B5-Shakespeare-Baseline/last.ckpt" "../model/MultiEpoch-1B5-Shakespeare-Baseline.pth"
!cd "{TRAINER_DIR}" && ls -alh "../model/MultiEpoch-1B5-Shakespeare-Baseline.pth"

## Stage 2 : Noisy model training

In [ ]:
# Start the foundation model training
!cd "{TRAINER_DIR}" && \
    export WANDB_MODE="{WANDB_MODE}" && \
    python new_train.py fit \
        -c "{NOTEBOOK_DIR}/MultiEpoch-1B5-instruct-noisy.yaml" \
        --trainer.logger.init_args.name="{WANDB_PREFIX} - Shakespeare Noisy (ctx=2048, {DEEPSPEED_STRAT})" \
        --trainer.strategy="{DEEPSPEED_STRAT}" \
        --trainer.devices="{GPU_DEVICES}" 

In [ ]:
# Lets export the model from the checkpoint
!cd "{TRAINER_DIR}" && \
    python export_checkpoint.py "../checkpoint/MultiEpoch-1B5-Shakespeare-Noisy/last.ckpt" "../model/MultiEpoch-1B5-Shakespeare-Noisy.pth"
!cd "{TRAINER_DIR}" && ls -alh "../model/MultiEpoch-1B5-Shakespeare-Noisy.pth"